In [1]:
import cv2
import pytesseract
import pandas as pd
import re
from PIL import Image
import os

In [2]:
# Replace 'your_receipt_image.jpg' with the path to your image file
#image_path = 'PXL_20240925_014423209~2v1.jpg' # clean image
image_path = 'PXL_20240925_014423209~2v1_HighContrast.jpg' # High contrast clean image
#image_path = 'PXL_20240925_014423209~2v1_HighContrastGimp1.png' # High contrast clean image
#image_path = 'PXL_20240923_014803753.MP~2.jpg' # -> image 4
#image_path = 'PXL_20240923_014546368~2.jpg' # -> image 3
#image_path = 'PXL_20240922_025356838~2.jpg' # -> image 2
#image_path = 'PXL_20240922_050714477~2.jpg' # -> image 1
filename , extn = os.path.splitext(image_path)
filepath = "C:\\Users\\sunil\\Desktop\\"
output_file_name = filename + '.txt'
output_file = filepath + output_file_name
print(output_file)
img = cv2.imread(image_path)

C:\Users\sunil\Desktop\PXL_20240925_014423209~2v1_HighContrast.txt


In [3]:
# Preprocessing: Convert to grayscale, blur, and threshold
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
_, binary_img = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Dilation to enhance the text
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
dilated_img = cv2.dilate(binary_img, kernel, iterations=1)

# Perform OCR
custom_config = r'--oem 3 --psm 6'
extracted_text = pytesseract.image_to_string(dilated_img, config=custom_config)

In [4]:
print(extracted_text)

You could win a $1000 GiftCard!
Visit survey.walmart.com#7TN4P910V6RF
For more details, see back of receipt.
]
Walmart ><
WM Supercenter
770-772-9033 Mar. JACKSON
5200 WINDWARD PKWY
ALPHARETTA GA 30004
ST# 02941 OP# 009006 TE# 06 TR# 00209
# ITEMS SOLD 9
TCH 8701 2652 2664 1811 5298
PET EMRE TF
Wl a Wf | i | li) |
A
WATERBOTTLES 840216539460. 3.00 xX
WATERBOTTLES 840216539580 3.00 x
BOUNCE SHT 037000971920 2.97 x
STRAWBERRIES 810343020740 F 2.96 R
BLUEBERRIES 812092030110 F 5.37 R
KIWI 2LB 732313002420 F 4.97 R
BLACK GRAPE 083477010320 F
2.340 lb. @ 1 Ib. /1.58 3.70 R
ORG BANANAS 681131098950 F
3.830 Ib. @ 1 Ib. /0.74 2.83 R
ORGANIC MLK 078742066270 F 6.98 Y
SUBTOTAL 35.78
TAX2 3.7500 % 1.014
TAX1 7.7500 % 0.70
TOTAL 37.49
DEBIT TEND 37.49
CHANGE DUE 0.00
EFT DEBIT PAY FROM PRIMARY
37.49 TOTAL PURCHASE
US DEBIT- 2997 I 0 REF # 426950552767
NETWORK ID. OOOL APPR. CODE 641579
AID A0000000042203
TERMINAL # 26919378
xNo Signature Required
09/24/24 21:43:00
|
[a] Oo Get free delivery
t °
"2

In [7]:
# Define regular expressions to capture the two formats of Walmart Headers

# Format 1: 3 lines Walmart Header
header_format_1 = re.compile(
    r"(You could win a \$1000 GiftCard!\nVisit survey\.walmart\.com#([A-Za-z0-9]+)\nFor more details, see back of receipt\.)", re.MULTILINE
)

# Format 2: 2 lines Walmart Header
header_format_2 = re.compile(
    r"(Give us feedback @ survey\.walmart\.com\nThank you! ID #:(\S+))", re.MULTILINE
)

# "Walmart Logo" rule (the line "Walmart ><")
walmart_logo_pattern = re.compile(r"Walmart ><", re.MULTILINE)

# Check for "Walmart Headers" and "Walmart Logo" and extract the Receipt ID and headers
receipt_id = None
header = None
header_contents = None

# Extract lines before the "Walmart Logo"
header_lines = []
remaining_text = extracted_text.split("\n")  # To store the remaining receipt text

for line in extracted_text.split("\n"):
    if walmart_logo_pattern.search(line):
        break  # Stop when we reach the "Walmart Logo"
    header_lines.append(line)

# Remove the detected "Walmart Headers" from the remaining text
header_block = "\n".join(header_lines)
remaining_text = "\n".join(remaining_text[len(header_lines)+1:])  # Skip Walmart Logo

# Check which header format matches and extract the Receipt ID
if header_format_1.search(header_block):
    header = "Format 1 (3 lines)"
    match = header_format_1.search(header_block)
    header_contents = match.group(1)
    receipt_id = match.group(2)
elif header_format_2.search(header_block):
    header = "Format 2 (2 lines)"
    match = header_format_2.search(header_block)
    header_contents = match.group(1)
    receipt_id = match.group(2)

# Print the Walmart Headers, Receipt ID, and Walmart Logo
if header:
    print(f"Walmart Headers Detected: {header}")
    print(f"Receipt ID: {receipt_id}")
    print("\nWalmart Header Content:")
    print(header_contents)
else:
    print("No Walmart Headers detected.")

# Check for "Walmart Logo" and print it
logo_match = walmart_logo_pattern.search(extracted_text)
if logo_match:
    print("\nWalmart Logo detected:")
    print("Walmart ><")

# After detection, we remove the header and logo from further use
print("\nRemaining Text after removing Walmart Headers and Walmart Logo:")
print(remaining_text)

# Step 3: Detect Store Details section (5 lines after the Walmart Logo)
store_details_lines = remaining_text.split("\n")[:5]  # Capture 5 lines after the logo

if len(store_details_lines) == 5:
    print("\nStore Details Section Detected:")
    
    # Line 1: Store type and name (e.g., "WM Supercenter")
    store_type_match = re.search(r"(Supercenter|Discount Store|Neighborhood Market|Fuel Station|Grocery Pickup|Deli)", store_details_lines[0])
    store_type = store_type_match.group(0) if store_type_match else "Unknown Store Type"
    print(f"Store Type: {store_type}")
    
    # Line 2: Store phone number and manager name
    phone_manager_match = re.search(r"(\d{3}-\d{3}-\d{4})\s+Mgr\.\s+([A-Za-z]+)", store_details_lines[1])
    if phone_manager_match:
        phone_number = phone_manager_match.group(1)
        manager_name = phone_manager_match.group(2)
        print(f"Phone Number: {phone_number}")
        print(f"Manager: {manager_name}")
    
    # Line 3: Street number and name
    print(f"Street Address: {store_details_lines[2]}")
    
    # Line 4: City, State, and ZIP code
    print(f"City, State, ZIP: {store_details_lines[3]}")
    
    # Line 5: ST#, OP#, TE#, TR# extraction
    line_5_match = re.search(r"ST#\s*(\d+)\s+OP#\s*(\d+)\s+TE#\s*(\d+)\s+TR#\s*(\d+)", store_details_lines[4])
    if line_5_match:
        store_number = line_5_match.group(1)
        operator_number = line_5_match.group(2)
        terminal_number = line_5_match.group(3)
        transaction_number = line_5_match.group(4)
        print(f"Store Number (ST#): {store_number}")
        print(f"Operator Number (OP#): {operator_number}")
        print(f"Terminal Number (TE#): {terminal_number}")
        print(f"Transaction Number (TR#): {transaction_number}")
    
    # Remove the "Store Details" section from the remaining text
    remaining_text = "\n".join(remaining_text.split("\n")[5:])
else:
    print("Store Details section not detected properly.")

# Step 4: Detect "# ITEMS SOLD" section and find number of items sold
items_sold_pattern = re.compile(r"# ITEMS SOLD\s+(\d+)", re.MULTILINE)
items_sold_match = items_sold_pattern.search(remaining_text)

if items_sold_match:
    items_sold = items_sold_match.group(1)
    print(f"\nCount of Items Purchased Detected: {items_sold} items sold.")
    
    # Remove this section from the remaining text
    remaining_text = items_sold_pattern.sub("", remaining_text)
else:
    print("\nCount of Items Purchased section not detected.")

# Step 5: Detect "Transaction #" section, allowing for common OCR errors (like TCH instead of TC#)
transaction_pattern = re.compile(r"T[CCH#]+\s+(\d{4}\s+\d{4}\s+\d{4}\s+\d{4}\s+\d{4})", re.MULTILINE)
transaction_match = transaction_pattern.search(remaining_text)

if transaction_match:
    transaction_number = transaction_match.group(1)
    print(f"\nTransaction # Detected: TC# {transaction_number}")
    
    # Remove this section from the remaining text
    remaining_text = transaction_pattern.sub("", remaining_text)
else:
    print("\nTransaction # section not detected.")

# Print remaining text after removing "Store Details", "Count of Items Purchased", and "Transaction #"
print("\nRemaining Text after removing Transaction #:")
print(remaining_text)

Walmart Headers Detected: Format 1 (3 lines)
Receipt ID: 7TN4P910V6RF

Walmart Header Content:
You could win a $1000 GiftCard!
Visit survey.walmart.com#7TN4P910V6RF
For more details, see back of receipt.

Walmart Logo detected:
Walmart ><

Remaining Text after removing Walmart Headers and Walmart Logo:
WM Supercenter
770-772-9033 Mar. JACKSON
5200 WINDWARD PKWY
ALPHARETTA GA 30004
ST# 02941 OP# 009006 TE# 06 TR# 00209
# ITEMS SOLD 9
TCH 8701 2652 2664 1811 5298
PET EMRE TF
Wl a Wf | i | li) |
A
WATERBOTTLES 840216539460. 3.00 xX
WATERBOTTLES 840216539580 3.00 x
BOUNCE SHT 037000971920 2.97 x
STRAWBERRIES 810343020740 F 2.96 R
BLUEBERRIES 812092030110 F 5.37 R
KIWI 2LB 732313002420 F 4.97 R
BLACK GRAPE 083477010320 F
2.340 lb. @ 1 Ib. /1.58 3.70 R
ORG BANANAS 681131098950 F
3.830 Ib. @ 1 Ib. /0.74 2.83 R
ORGANIC MLK 078742066270 F 6.98 Y
SUBTOTAL 35.78
TAX2 3.7500 % 1.014
TAX1 7.7500 % 0.70
TOTAL 37.49
DEBIT TEND 37.49
CHANGE DUE 0.00
EFT DEBIT PAY FROM PRIMARY
37.49 TOTAL PURCHASE
US D

In [8]:


# Extract text before "SUBTOTAL"
extracted_text2 = remaining_text.split("SUBTOTAL")[0].strip()

# Output the result
print(extracted_text2)


PET EMRE TF
Wl a Wf | i | li) |
A
WATERBOTTLES 840216539460. 3.00 xX
WATERBOTTLES 840216539580 3.00 x
BOUNCE SHT 037000971920 2.97 x
STRAWBERRIES 810343020740 F 2.96 R
BLUEBERRIES 812092030110 F 5.37 R
KIWI 2LB 732313002420 F 4.97 R
BLACK GRAPE 083477010320 F
2.340 lb. @ 1 Ib. /1.58 3.70 R
ORG BANANAS 681131098950 F
3.830 Ib. @ 1 Ib. /0.74 2.83 R
ORGANIC MLK 078742066270 F 6.98 Y


In [9]:
# Define regex patterns
single_line_pattern = re.compile(r'^.*\d{10,12}.*\d+\.\d{2}.*$')
#weight_item_pattern = re.compile(r'^\d+\.\d{3}\s*lb\.\s*@\s*\d+\s*lb\.|oz.*\d+\.\d{2}.*$')
weight_item_pattern = re.compile(r'^\d+\.\d{3}\s*[lI]b\.\s*@\s*\d+\s*[lI]b\.|oz.*\d+\.\d{2}.*$')


# Split the text into lines
lines = extracted_text2.strip().split('\n')

# Initialize lists to store categorized items
one_line_items = []
two_line_items = []

# Loop through the lines and categorize items
i = 0
while i < len(lines):
    line = lines[i].strip()
    
    # Check if the line matches a single-line item
    if single_line_pattern.match(line):
        one_line_items.append(line)
    
    # Check if it's the start of a two-line item (non-priced weighted items)
    elif i + 1 < len(lines) and weight_item_pattern.match(lines[i + 1]):
        two_line_items.append(f"{line}\n{lines[i + 1]}")
        i += 1  # Skip the next line since it's part of the two-line item
    
    i += 1

# Output the results
print("Items with complete information in one line:")
for item in one_line_items:
    print(item)

print("\nItems with information spanning two lines:")
for item in two_line_items:
    print(item)

Items with complete information in one line:
WATERBOTTLES 840216539460. 3.00 xX
WATERBOTTLES 840216539580 3.00 x
BOUNCE SHT 037000971920 2.97 x
STRAWBERRIES 810343020740 F 2.96 R
BLUEBERRIES 812092030110 F 5.37 R
KIWI 2LB 732313002420 F 4.97 R
ORGANIC MLK 078742066270 F 6.98 Y

Items with information spanning two lines:
BLACK GRAPE 083477010320 F
2.340 lb. @ 1 Ib. /1.58 3.70 R
ORG BANANAS 681131098950 F
3.830 Ib. @ 1 Ib. /0.74 2.83 R


In [10]:
# The new regex to extract data from one_line_items
one_line_regex = re.compile(r"([A-Za-z\s]+[a-zA-Z0-9\s]+)\s+(\d{10,12}.)\s*([FR]?)\s+(\d+\.\d{2})\s*([xXR]?)")

print("Extracted Information from one-line items:")
for item in one_line_items:
    match = one_line_regex.match(item)
    if match:
        item_name = match.group(1)
        item_code = match.group(2)
        optional_FR = match.group(3)
        price = match.group(4)
        optional_XR = match.group(5)
        
        # Display the extracted information
        print(f"Item: {item_name}, Code: {item_code}, F/R: {optional_FR}, Price: {price}, X/R: {optional_XR}")

Extracted Information from one-line items:
Item: WATERBOTTLES, Code: 840216539460., F/R: , Price: 3.00, X/R: x
Item: WATERBOTTLES, Code: 840216539580, F/R: , Price: 3.00, X/R: x
Item: BOUNCE SHT, Code: 037000971920, F/R: , Price: 2.97, X/R: x
Item: STRAWBERRIES, Code: 810343020740 , F/R: F, Price: 2.96, X/R: R
Item: BLUEBERRIES, Code: 812092030110 , F/R: F, Price: 5.37, X/R: R
Item: KIWI 2LB, Code: 732313002420 , F/R: F, Price: 4.97, X/R: R
Item: ORGANIC MLK, Code: 078742066270 , F/R: F, Price: 6.98, X/R: 


In [11]:
# New regex for two-line items
combined_regex = re.compile(
    r"([A-Za-z\s]+[A-Za-z])\s+(\d{10,12})\s*([FR]?)\s+(\d+\.\d{3})\s*(lb|Ib)\s*@\s*(\d+)\s*(lb|Ib)\s*/\s*(\d+\.\d{2})\s*(\d+\.\d{2})\s*([XR]?)"
)
two_line_items_list = []
for item in two_line_items:
    
    item_combined = item.replace('\n', ' ')
    two_line_items_list.append(item_combined)
    print(f"Processing item: {item_combined}")  # To inspect the exact format after combining lines

Processing item: BLACK GRAPE 083477010320 F 2.340 lb. @ 1 Ib. /1.58 3.70 R
Processing item: ORG BANANAS 681131098950 F 3.830 Ib. @ 1 Ib. /0.74 2.83 R


In [12]:
for item in two_line_items_list:
    print(item)


BLACK GRAPE 083477010320 F 2.340 lb. @ 1 Ib. /1.58 3.70 R
ORG BANANAS 681131098950 F 3.830 Ib. @ 1 Ib. /0.74 2.83 R


In [13]:

# print the text for weighted items
for i in two_line_items_list:
    print(i)
# Debugging: Print the text we're analyzing
print("\nText for Analysis:")
for i, line in enumerate(two_line_items_list):
    print(f"Line {i+1}: {line}")

# Step 1: New simplified regex pattern to capture item name, item code, and price details
# Adjusting the pattern to handle price per unit and total price correctly
weighted_item_pattern = re.compile(
    r"([A-Za-z\s]+)\s+(\d{10,12})\s*([FR]?)\s+(\d+\.\d{3}\s+(lb|Ib)\.|oz\.)\s+@\s+1\s+(lb|Ib)\.\s+\/\s*(\d+\.\d{2})\s+(\d+\.\d{2})\s*([XR]?)"
)

# Find all weighted items from the list of lines
weighted_items = []
for line in two_line_items_list:
    match = weighted_item_pattern.findall(line)
    if match:
        weighted_items.extend(match)

# Step 2: Display the weighted item list
print("\nWeighted Item List:")
if not weighted_items:
    print("No weighted items found.")
else:
    for item in weighted_items:
        item_name = item[0].strip()
        item_code = item[1]
        optional_code = item[2]
        units_purchased_vs_price_per_unit = f"{item[3]} @ 1 lb. / {item[4]}"  # Combine weight and price per unit
        total_price = item[5]
        price_type = item[6]
        print(f"Item: {item_name}, Code: {item_code}, Units Purchased vs Price per Unit: {units_purchased_vs_price_per_unit}, Total Price: {total_price}, Price Type: {price_type}")

# Step 3: Additional debugging if needed
if not weighted_items:
    print("\nNo matches were found. Let's debug:")
    for line in two_line_items_list:
        print(f"\nAnalyzing line: {line}")
        for match in re.finditer(r"\d{10,12}", line):
            print(f"Item Code match at position {match.start()}: {match.group(0)}")
        for match in re.finditer(r"\d+\.\d{3}\s+(lb\.|oz\.)", line):
            print(f"Weight match at position {match.start()}: {match.group(0)}")
        for match in re.finditer(r"@\s+1\s+lb\.\s+\/\s*\d+\.\d{2}", line):
            print(f"Units Purchased vs Price per Unit match at position {match.start()}: {match.group(0)}")
        for match in re.finditer(r"\d+\.\d{2}", line):
            print(f"Total Price match at position {match.start()}: {match.group(0)}")

BLACK GRAPE 083477010320 F 2.340 lb. @ 1 Ib. /1.58 3.70 R
ORG BANANAS 681131098950 F 3.830 Ib. @ 1 Ib. /0.74 2.83 R

Text for Analysis:
Line 1: BLACK GRAPE 083477010320 F 2.340 lb. @ 1 Ib. /1.58 3.70 R
Line 2: ORG BANANAS 681131098950 F 3.830 Ib. @ 1 Ib. /0.74 2.83 R

Weighted Item List:
Item: BLACK GRAPE, Code: 083477010320, Units Purchased vs Price per Unit: 2.340 lb. @ 1 lb. / lb, Total Price: Ib, Price Type: 1.58
Item: ORG BANANAS, Code: 681131098950, Units Purchased vs Price per Unit: 3.830 Ib. @ 1 lb. / Ib, Total Price: Ib, Price Type: 0.74
